In [28]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import talib

# Load and visualize data.

In [ ]:
spy_data = pd.read_csv('data/SPY.csv', header=0, names=['Date', 'Open', 'High', 'Low', 'Close', 'AdjClose', 'Volume'])

In [ ]:
spy_data[35:1000]

In [ ]:
count = 0
coke_data = pd.read_csv('data/coke.csv', header=0, names = ['datadate', 'tic', 'cshtrd', 'prccd', 'prchd', 'prcld', 'prcod']).dropna()


In [ ]:
# Process data.
coke_data['datadate'] = pd.to_datetime(coke_data['datadate'], format='%Y%m%d')
coke_data['prccd'] = pd.to_numeric(coke_data['prccd']) #closing price
coke_data['prcod'] = pd.to_numeric(coke_data['prcod']) #opening price
coke_data['prchd'] = pd.to_numeric(coke_data['prchd']) 
coke_data['prcld'] = pd.to_numeric(coke_data['prcld'])
coke_data['cshtrd'] = pd.to_numeric(coke_data['cshtrd'])
coke_data

In [ ]:
coke_data.plot('datadate', 'prcod', title='Coke opening stock price from 2006-2021', grid=True)

In [ ]:
coke_data.plot('datadate', 'prccd', title='Coke closing stock price from 2006-2021', grid=True)

# Compute technical indicators.

In [ ]:
#RSI, MACD, OBV, CCI, ADI
coke_data['rsi'] = talib.RSI(coke_data['prcod']) #default period of 14 days
coke_data['rsi'][:15]

In [ ]:
coke_data.plot('datadate', 'rsi', title='Relative Strength Index 2006-2021', grid=True)

In [ ]:
coke_data['macd'], coke_data['macdsignal'], coke_data['macdhist'] = talib.MACD(coke_data['prccd']) # defaults: fastperiod=12, slowperiod=26, signalperiod=9

In [ ]:
coke_data.plot('datadate', ['macd', 'macdsignal', 'macdhist'], title='Moving Average Convergence/Divergence 2006-2021', grid=True)

In [ ]:
coke_data['obv'] = talib.OBV(coke_data['prccd'], coke_data['cshtrd'])
coke_data.plot('datadate', 'obv', title='On-Balance Volume 2006-2021', grid=True)

In [ ]:
coke_data['cci'] = talib.CCI(coke_data['prchd'], coke_data['prcld'], coke_data['prccd']) #default period: 14 days
coke_data.plot('datadate', 'cci', title='Commodity Channel Index 2006-2021', grid=True)

In [ ]:
coke_data['adx'] = talib.ADX(coke_data['prchd'], coke_data['prcld'], coke_data['prccd']) #default period: 14 days
coke_data.plot('datadate', 'adx', title='Average Directional Index 2006-2021', grid=True)

In [25]:
all_data = pd.read_csv('data/allStocks.csv', header=0, names=['gvkey', 'iid', 'Date', 'tic', 
                                                              'Volume', 'Close', 'High', 'Low', 'Open'])

/home/saad/envs/stocks/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [19]:
stock = []
for e in all_data['tic'].unique():
    if '.' in e or len(e) > 4:
        continue
    stock.append(e)

In [23]:
filtered = all_data.loc[all_data['tic'].isin(stock)]

In [24]:
filtered.to_csv('data/filtered.csv', index=0)

In [29]:
fp = pd.read_csv('data/filtered.csv', header=0)
fp

,gvkey,iid,Date,tic,Volume,Close,High,Low,Open
0,1045,2,20060103,AAR,14000.0,20.01,20.400,20.0000,20.25
1,1045,2,20060104,AAR,7200.0,20.10,20.140,19.9500,20.00
2,1045,2,20060105,AAR,17700.0,20.12,20.400,20.0100,20.10
3,1045,2,20060106,AAR,7600.0,20.34,20.340,20.1200,20.12
4,1045,2,20060109,AAR,21500.0,20.46,20.490,20.1500,20.49
...,...,...,...,...,...,...,...,...,...
1876119,316056,1,20211011,ALLE,510492.0,129.10,131.980,129.0700,130.97
1876120,316056,1,20211012,ALLE,533976.0,128.36,130.605,127.9900,129.40
1876121,316056,1,20211013,ALLE,590693.0,128.92,130.050,127.8600,128.70
1876122,316056,1,20211014,ALLE,472197.0,132.36,132.590,128.7201,130.09


In [32]:
# generate dataframe with technical indicators

output = []

for s in fp['tic'].unique():
    # select all rows with data
    df = fp.loc[fp['tic'] == s]
    # compute technical indicator
    df['rsi'] = talib.RSI(df['Open'])
    df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Close']) # defaults: fastperiod=12, slowperiod=26, signalperiod=9
    df['obv'] = talib.OBV(df['Close'], df['Volume'])
    df['cci'] = talib.CCI(df['High'], df['Low'], df['Close']) #default period: 14 days
    df['adx'] = talib.ADX(df['High'], df['Low'], df['Close']) #default period: 14 days
    
    output.append(df)
    
output_df = pd.concat(output)
output_df[output_df['tic'] == 'AAR']

/home/saad/envs/stocks/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/saad/envs/stocks/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/saad/envs/stocks/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

,gvkey,iid,Date,tic,Volume,Close,High,Low,Open,rsi,macd,macdsignal,macdhist,obv,cci,adx
0,1045,2,20060103,AAR,14000.0,20.01,20.4000,20.00,20.25,NaN,NaN,NaN,NaN,14000.0,NaN,NaN
1,1045,2,20060104,AAR,7200.0,20.10,20.1400,19.95,20.00,NaN,NaN,NaN,NaN,21200.0,NaN,NaN
2,1045,2,20060105,AAR,17700.0,20.12,20.4000,20.01,20.10,NaN,NaN,NaN,NaN,38900.0,NaN,NaN
3,1045,2,20060106,AAR,7600.0,20.34,20.3400,20.12,20.12,NaN,NaN,NaN,NaN,46500.0,NaN,NaN
4,1045,2,20060109,AAR,21500.0,20.46,20.4900,20.15,20.49,NaN,NaN,NaN,NaN,68000.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1508,1045,2,20111228,AAR,34225.0,4.71,4.9100,4.61,4.61,33.499023,-1.169541,-1.465642,0.296101,-1515574.0,-54.440796,42.517337
1509,1045,2,20111229,AAR,35487.0,4.68,4.7501,4.67,4.67,34.301181,-1.102216,-1.392957,0.290741,-1551061.0,-62.813924,41.680403
1510,1045,2,20111230,AAR,172850.0,4.49,4.4900,4.10,4.41,32.473232,-1.052064,-1.324778,0.272714,-1723911.0,-134.998289,41.525902
1511,1045,2,20120103,AAR,131214.0,4.65,4.6600,4.45,4.50,33.788583,-0.988018,-1.257426,0.269408,-1592697.0,-67.060178,40.994909


In [33]:
output_df.to_csv('data/filtered_with_ti.csv', index=0)